In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk==3.7
!pip install -U Flask
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import hashlib

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# client = storage.Client()
# bucket_name = 'ir_assg3_eithan'

# bucket = client.bucket(bucket_name)

# # Get a list of all the blobs in the folder
# blobs = bucket.list_blobs(prefix='indexes/')

# # Iterate over the list of blobs and download each one to a local file
# for blob in blobs:
#     if 'doc_norm_dict.pickle' in blob.name or 'doc_len_dict.pickle' in blob.name:
#         local_file_name = blob.name[blob.name.rfind("/")+1:]
#         blob.download_to_filename(local_file_name)

In [3]:
from flask import Flask, request, jsonify
import collections
import csv
import gzip
import math
import json

!pip install fasttext
import fasttext
import requests

!pip install gensim
# from gensim.models.wrappers import * 

In [4]:
%cd /home/dataproc
from inverted_index_gcp import InvertedIndex
# from inverted_index_gcp import *
# import inverted_index_gcp as iig
from inverted_index_gcp import * 

/home/dataproc


In [5]:

# import importlib
# importlib.reload(iig)

In [6]:
path_anchor = "/home/dataproc/postings_gcp_anchor"
path_body = "/home/dataproc/postings_gcp_body"
path_title = "/home/dataproc/postings_gcp_title"

In [7]:
# Read indexes
body_index = InvertedIndex.read_index('.',"body_index")
anchor_index = InvertedIndex.read_index('.',"anchor_index")
title_index = InvertedIndex.read_index('.',"title_index")

In [8]:
body_index.bin_path = path_body
anchor_index.bin_path = path_anchor
title_index.bin_path = path_title

In [9]:
with open('doc_id_title_dict.pickle','rb') as f:
    doc_title_dict = pickle.load(f)
    
with open('doc_len_dict.pickle','rb') as f:
    doc_len_dict = pickle.load(f)
    
with open('page_views_dict.pkl','rb') as f:
    page_views_dict = pickle.load(f)

with open('page_rank_dict.pickle','rb') as f:
    page_rank_dict = pickle.load(f)

with open('doc_norm_dict.pickle','rb') as f:
    doc_norm_dict = pickle.load(f)

In [10]:
with open('queries control group.json', 'rt') as f:
    queries_control_group = json.load(f)

with open('test_queries.json', 'rt') as f:
    queries_train = json.load(f)

In [11]:
from contextlib import closing
def read_posting_list(inverted, w, index_type=""):
    with closing(MultiFileReader()) as reader:
        locs = inverted.posting_locs[w]
        b = reader.read(locs, inverted.df[w] * TUPLE_SIZE,index_type)
        posting_list = []
        for i in range(inverted.df[w]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
        return posting_list

In [12]:
# Reading posting list example:
# lst = read_posting_list(body_index,"texas", path_body)
# print(len(lst))

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = set(tok for tok in tokens if tok not in all_stopwords)
    return tokens



In [39]:



def normalize(tokens):
    counter = Counter(tokens)
    norm = 0
    for value in counter.values():
        norm += value*value
    if norm == 0 :
        return 0
    return 1/(math.sqrt(norm))


def tf_idf_calc (token, doc_id, tf):
    tf =  tf/doc_len_dict[doc_id]
#     idf = math.log(N/app.body_index.df[token], 2)
    idf = math.log(N/body_index.df[token], 2)
    return tf*idf

def bm25_update(token, doc_id, tf):
    k1 = 1.2
    B = 0.75
    N = 6348910
    avgl = 319.5242353411845
    idf = math.log(((N - body_index.df[token] + 0.5)/(body_index.df[token]+0.5)) + 1, math.e)
    score = (idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - B + B * (doc_len_dict[doc_id] / avgl)))))
    return score


def top_n_results(similarity_dict, res, n=100):
    # Top up to 100
    res_size = min(n, len(similarity_dict.keys()))
    top100 = list(sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)[:res_size])
    # Returning the top up to 100 tuples -> (ID, Tittle)
    for pair in top100:
        res.append((pair[0], doc_title_dict[pair[0]]))

In [106]:
#Global Variables: 

N = 6348910
page_views_avg = sum(page_views_dict.values())/len(page_views_dict.values())
page_rank_avg = sum(page_rank_dict.values())/len(page_rank_dict.values())
page_rank_max = max(page_rank_dict.values())
page_views_max = max(page_views_dict.values())

page_rank_min = min(page_rank_dict.values())

In [101]:
print(page_rank_avg)
print(page_rank_max)
print(list(sorted(page_rank_dict.items(), key=lambda item:item[1], reverse=True)[:50]))

1.0000000000249587
9913.728782160773
[(3434750, 9913.728782160773), (10568, 5385.349263642041), (32927, 5282.081575765278), (30680, 5128.233709604123), (5843419, 4957.567686263867), (68253, 4769.2782653551585), (31717, 4486.350180548308), (11867, 4146.414650912768), (14533, 3996.4664408855006), (645042, 3531.6270898037424), (17867, 3246.0983906041424), (5042916, 2991.9457391661776), (4689264, 2982.324883041748), (14532, 2934.7468292031704), (25391, 2903.546223513399), (5405, 2891.4163291546365), (4764461, 2834.3669873326626), (15573, 2783.865118158838), (9316, 2782.0396464137707), (8569916, 2775.286191840016), (606848, 2773.576318485695), (14653, 2695.429324828789), (18618239, 2450.113149559415), (19344515, 2347.0140777778715), (26667, 2224.0802443826105), (23366462, 2143.515050566466), (64065, 2127.763802350797), (22936, 2127.075241657329), (26779, 2116.152456234464), (5407, 2099.6738846846097), (19827221, 2092.4513474706423), (43432956, 2085.7409097922205), (679, 2065.0308793052063),

In [15]:
def search_title():
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE TITLE of articles, ordered in descending order of the NUMBER OF 
        QUERY WORDS that appear in the title. For example, a document with a 
        title that matches two of the query words will be ranked before a 
        document with a title that matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_title?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''

    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter = collections.Counter()
    for token in tokenize(query):
        for docId_tf in title_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter.most_common()))
    # END SOLUTION
    return res

In [54]:
print(search_title())

[(970755, 'LinkedIn'), (27769500, 'LinkedIn Pulse'), (36070366, '2012 LinkedIn hack'), (41726116, 'LinkedIn Learning'), (50191962, 'Timeline of LinkedIn'), (57147095, 'LinkedIn Top Companies'), (62976368, 'HiQ Labs v. LinkedIn')]


In [91]:
def search_anchor():
    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter_anchor = collections.Counter()
    for token in tokenize(query):
        for docId_tf in anchor_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter_anchor[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter_anchor.most_common()))
    # END SOLUTION
    return res

In [92]:
print(search_anchor())

[(324, 'Academy Awards'), (358, 'Algeria'), (590, 'Austin (disambiguation)'), (679, 'Animal (disambiguation)'), (736, 'Albert Einstein'), (803, 'Arabic'), (849, 'Aircraft'), (924, 'A. A. Milne'), (1078, 'Antisemitism'), (1148, 'Adelaide'), (1171, 'Abbreviation'), (1528, 'Ahmed II'), (1921, 'Al-Qaeda'), (1950, 'American Civil Liberties Union'), (2007, 'Archery'), (2273, 'AFC Ajax'), (2425, 'Aurangzeb'), (3138, 'Atlanta'), (3165, 'ACF Fiorentina'), (3343, 'Belgium'), (3383, 'Brazil'), (3416, 'Bryozoa'), (3462, 'Bolivia'), (3708, 'Brussels'), (3956, 'Badminton'), (4092, 'Bishop'), (4293, 'Baltimore Ravens'), (4566, 'Baku'), (4620, 'Bronze Age'), (4700, 'Bessel function'), (4887, 'British Army'), (5363, 'Video game'), (5399, 'Colorado'), (5405, 'China'), (5407, 'California'), (5551, 'Costa Rica'), (5593, 'Cyprus'), (5717, 'Canary Islands'), (5865, 'Council of Europe'), (5882, 'Cardiff'), (5902, 'Capital punishment'), (5992, 'Traditional Chinese medicine'), (6325, 'Church (building)'), (664

In [16]:
def get_pageview():
    ''' Returns the number of page views that each of the provide wiki articles
        had in August 2021.

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pageview
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pageview', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    '''

    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 21105323, 23536538, 23797577]
    if len(wiki_ids) == 0:
        return res
    # BEGIN SOLUTION
    
    res = [page_views_dict.get(doc_id, -1) for doc_id in wiki_ids]

    # END SOLUTION
    return res

In [62]:
print(get_pageview())

[84370, 1042, 30483, 38197]


In [34]:
def get_pagerank():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 3434750, 32927]
    if len(wiki_ids) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    res = [page_rank_dict.get(doc_id, -1) for doc_id in wiki_ids]
    # END SOLUTION
#     return jsonify(res)
    return res

In [35]:
# print(get_pagerank())

[50.7020900020747, 9913.728782160773, 5282.081575765278]


In [19]:
def top_n_results(similarity_dict, res, n=100):
    # Top up to 100
    res_size = min(n, len(similarity_dict.keys()))
    top100 = list(sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)[:res_size])
    # Returning the top up to 100 tuples -> (ID, Tittle)
    for pair in top100:
        res.append((pair[0], doc_title_dict[pair[0]]))

In [20]:
def search_body():
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
#     query = request.args.get('query', '')
    query = 'air jordan'
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    similarity_dict = {}
    tokens = tokenize(query)
    for token in tokens:
        for doc_tf in body_index.read_posting_list(token):
            if doc_tf[0] not in similarity_dict:
                similarity_dict[doc_tf[0]] = 0
            similarity_dict[doc_tf[0]] += tf_idf_calc(token, doc_tf[0], doc_tf[1])

    for doc in similarity_dict.keys():
        similarity_dict[doc] = similarity_dict[doc]*normalize(tokens)*doc_norm_dict[doc]
        
#     top100 = list(sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)[:100])
    
#     for pair in top100:
#         res.append((pair[0], doc_title_dict[pair[0]]))
    top_n_results(similarity_dict, res, 100)

    # END SOLUTION
#     return jsonify(res)
#     return res
    return res




In [21]:
print(search_body())

[(59805922, 'North Region, Jordan'), (59986028, 'Central Region, Jordan'), (59986033, 'South Region, Jordan'), (59806162, 'Kufranjah'), (59986330, 'Al-Judayyda'), (59986333, 'Al-Qūaīrah'), (59986355, 'Al-Kūrah'), (59986357, 'Al-Wasṭīyah'), (59986359, 'Aṭ-Ṭaībah'), (59986392, 'Al-Qaṣr'), (59986393, 'Al-Qaṭrāneh'), (59986394, "'Ayy"), (59986395, "Faqū'e"), (59986405, "Al-Betrā'"), (59986419, 'Al-Ḥuseīniyah'), (59986538, 'Al-Hasa District'), (59986541, 'Bṣaīrā'), (59986543, 'Al-Hāshimiyah District'), (59986329, 'Uhod area'), (29082116, 'Jordan Gabriel'), (59986298, "Qaṣabah 'Ajlūn"), (59986336, "Qaṣabah al-'Aqabah"), (59986346, 'Al-Āghwār ash-Shamāliyah'), (59986356, 'Al-Mazār ash-Shamālī'), (59986360, 'Banī Kenānah'), (59986361, "Banī 'Obeīd"), (59986369, 'Qaṣabah Jarash'), (59986375, 'Al-Āghwār al-Janūbī'), (59986391, 'Al-Mazār al-Janūbī'), (59986396, 'Qaṣabah al-Karak'), (59986422, "Qaṣabah Ma'ān"), (59986433, "Qaṣabah Ma'dabā"), (59986441, 'Badiah Shamaliyah'), (59986458, 'Badiah Ghar

In [13]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.bin
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip

--2023-01-11 11:41:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  38.2MB/s    in 4m 10s  

2023-01-11 11:46:08 (39.5 MB/s) - ‘wiki.en.zip’ saved [10356881291/10356881291]



In [2]:
%cd /home/dataproc

/home/dataproc


In [120]:
with open('new_train.json', 'rt') as f:
    new_queries = json.load(f)

In [131]:


def search(q, weights):
# def search(q):
    ''' Returns up to a 100 of your best search results for the query. This is 
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the 
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use 
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
#     query = request.args.get('query', '')
#     query = 'best marvel movie'
    query = q
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    similarity_dict = {}

    # Constants
    weights = [0.46162036, 1.42513415, 1.13945286, 1.87296152, 1.39693174]
    BODY_WEIGHT, ANCHOR_WEIGHT, TITLE_WEIGHT, PAGE_RANK_WEIGHT, PAGE_VIEW_WEIGHT = weights

    if query[-1] == '?':
        BODY_WEIGHT *= 2
    
    
    tokens = tokenize(query)
    
    tokens_in_doc_title = collections.Counter()
    tokens_in_doc_anchor = collections.Counter()
    
    for token in tokens:

        # Searching the term in anchor index
        if anchor_index.df.get(token, None):
            for doc_tf in anchor_index.read_posting_list(token):
                tokens_in_doc_anchor[doc_tf[0]] += 1
                similarity_dict[doc_tf[0]] = similarity_dict.get(doc_tf[0], 0) + math.pow(ANCHOR_WEIGHT, tokens_in_doc_anchor[doc_tf[0]])

        # Searching the term in title index
        if title_index.df.get(token, None):
            for doc_tf in title_index.read_posting_list(token):
                tokens_in_doc_title[doc_tf[0]] += 1
                similarity_dict[doc_tf[0]] = similarity_dict.get(doc_tf[0], 0) + math.pow(TITLE_WEIGHT, tokens_in_doc_title[doc_tf[0]])

        # Searching the term in body index
        if body_index.df.get(token, None):
            for doc_tf in body_index.read_posting_list(token):
                c = bm25_update(token, doc_tf[0], doc_tf[1]) * BODY_WEIGHT
                similarity_dict[doc_tf[0]] = similarity_dict.get(doc_tf[0], 0) + c

        # Adding page_rank and page_views of each relevant document
        for doc_id in similarity_dict.keys():
            similarity_dict[doc_id] += page_rank_dict.get(doc_id, 0) / page_rank_max * PAGE_RANK_WEIGHT
            similarity_dict[doc_id] += page_views_dict.get(doc_id, 0) / page_views_max * PAGE_VIEW_WEIGHT
            
    top_n_results(similarity_dict, res, 100)
    # END SOLUTION
#     return jsonify(res)
#     return top100
    return res


In [118]:
qs_res = []
for q, true_wids in new_queries.items():
    duration, ap = None, None
    t_start = time()
#     try:
    res = search(q)
    duration = time() - t_start
    pred_wids= [x[0] for x in res]
    ap = average_precision(true_wids, pred_wids)
#     except:
#         pass

    qs_res.append((q, duration, ap))

In [119]:
# # anchor w = 1.6 with Normalized Page Rank
avg = sum(x[2] for x in qs_res)/len(qs_res)
print(avg)
avg_duration = sum(x[1] for x in qs_res)/len(qs_res)
print(avg_duration)
print(qs_res)

0.2786333333333333
2.6805486122767133
[('best marvel movie', 5.743777751922607, 0.071), ('How do kids come to world?', 6.980289697647095, 0.031), ('Information retrieval', 1.3478457927703857, 0.814), ('LinkedIn', 0.018994808197021484, 0.58), ('How to make coffee?', 2.5993967056274414, 0.149), ('Ritalin', 0.0010020732879638672, 0.862), ('How to make wine at home?', 5.8004655838012695, 0.041), ('Most expensive city in the world', 10.753187656402588, 0.276), ('India', 1.3024449348449707, 0.227), ('how to make money fast?', 4.375547409057617, 0.183), ('Netflix', 0.09640121459960938, 0.288), ('Apple computer', 1.1648614406585693, 0.241), ('The Simpsons', 0.04111766815185547, 0.155), ('World cup', 8.145272493362427, 0.049), ('How to lose weight?', 0.7390344142913818, 0.438), ('Java', 0.11508655548095703, 0.295), ('Air Jordan', 2.0500125885009766, 0.098), ('how to deal with depression?', 1.0061063766479492, 0.148), ('How do you make gold', 3.6153953075408936, 0.0), ('Marijuana', 0.03350615501

In [104]:
# # anchor w = 1.6 and pr_w=3 
avg = sum(x[2] for x in qs_res)/len(qs_res)
print(avg)
avg_duration = sum(x[1] for x in qs_res)/len(qs_res)
print(avg_duration)
print(qs_res)

0.4663666666666667
2.544266446431478
[('best marvel movie', 5.320261240005493, 0.384), ('How do kids come to world?', 6.613449335098267, 0.204), ('Information retrieval', 1.295654058456421, 0.913), ('LinkedIn', 0.016718626022338867, 0.879), ('How to make coffee?', 2.422734498977661, 0.233), ('Ritalin', 0.0009181499481201172, 0.862), ('How to make wine at home?', 5.558455467224121, 0.318), ('Most expensive city in the world', 10.228096723556519, 0.372), ('India', 1.2321791648864746, 0.502), ('how to make money fast?', 4.142963171005249, 0.694), ('Netflix', 0.09411239624023438, 0.572), ('Apple computer', 1.095952033996582, 0.632), ('The Simpsons', 0.03714489936828613, 0.185), ('World cup', 7.819543123245239, 0.133), ('How to lose weight?', 0.7051734924316406, 0.461), ('Java', 0.11125016212463379, 0.441), ('Air Jordan', 1.9753596782684326, 0.494), ('how to deal with depression?', 0.9680581092834473, 0.226), ('How do you make gold', 3.422649383544922, 0.25), ('Marijuana', 0.032908439636230

In [78]:
# anchor w = 1.6
avg = sum(x[2] for x in qs_res)/len(qs_res)
print(avg)
avg_duration = sum(x[1] for x in qs_res)/len(qs_res)
print(avg_duration)
print(qs_res)

0.47203333333333336
2.5358163118362427
[('best marvel movie', 5.432099103927612, 0.433), ('How do kids come to world?', 6.597549676895142, 0.205), ('Information retrieval', 1.2853686809539795, 0.913), ('LinkedIn', 0.017248868942260742, 0.879), ('How to make coffee?', 2.444232225418091, 0.233), ('Ritalin', 0.0009329319000244141, 0.862), ('How to make wine at home?', 5.57279372215271, 0.34), ('Most expensive city in the world', 10.275249719619751, 0.293), ('India', 1.1861114501953125, 0.5), ('how to make money fast?', 4.123258113861084, 0.697), ('Netflix', 0.09115099906921387, 0.572), ('Apple computer', 1.1125831604003906, 0.629), ('The Simpsons', 0.03746461868286133, 0.185), ('World cup', 7.753355979919434, 0.167), ('How to lose weight?', 0.6946687698364258, 0.461), ('Java', 0.1112818717956543, 0.439), ('Air Jordan', 1.9457812309265137, 0.495), ('how to deal with depression?', 0.9309649467468262, 0.227), ('How do you make gold', 3.3989098072052, 0.333), ('Marijuana', 0.03276371955871582

In [89]:
# anchor w = 1.6 with question words body_w *1.75
avg = sum(x[2] for x in qs_res)/len(qs_res)
print(avg)
avg_duration = sum(x[1] for x in qs_res)/len(qs_res)
print(avg_duration)
print(qs_res)

0.4583666666666666
2.5249850670496623
[('best marvel movie', 5.390177488327026, 0.433), ('How do kids come to world?', 6.579817295074463, 0.223), ('Information retrieval', 1.28251314163208, 0.913), ('LinkedIn', 0.017787933349609375, 0.879), ('How to make coffee?', 2.4156410694122314, 0.235), ('Ritalin', 0.0009551048278808594, 0.862), ('How to make wine at home?', 5.5091469287872314, 0.246), ('Most expensive city in the world', 10.274772644042969, 0.293), ('India', 1.2007339000701904, 0.5), ('how to make money fast?', 4.1073877811431885, 0.665), ('Netflix', 0.09031319618225098, 0.572), ('Apple computer', 1.085829257965088, 0.629), ('The Simpsons', 0.03646349906921387, 0.185), ('World cup', 7.6935319900512695, 0.167), ('How to lose weight?', 0.69608473777771, 0.473), ('Java', 0.10883736610412598, 0.439), ('Air Jordan', 1.9532966613769531, 0.495), ('how to deal with depression?', 0.9420382976531982, 0.224), ('How do you make gold', 3.4022371768951416, 0.0), ('Marijuana', 0.032185316085815

In [85]:
# anchor w = 1.6 with question words body_w *1.5
avg = sum(x[2] for x in qs_res)/len(qs_res)
print(avg)
avg_duration = sum(x[1] for x in qs_res)/len(qs_res)
print(avg_duration)
print(qs_res)

0.45696666666666663
2.526063410441081
[('best marvel movie', 5.357511758804321, 0.433), ('How do kids come to world?', 6.538959264755249, 0.207), ('Information retrieval', 1.2849493026733398, 0.913), ('LinkedIn', 0.01724100112915039, 0.879), ('How to make coffee?', 2.4149649143218994, 0.239), ('Ritalin', 0.0009293556213378906, 0.862), ('How to make wine at home?', 5.493225574493408, 0.161), ('Most expensive city in the world', 10.22467041015625, 0.293), ('India', 1.1974902153015137, 0.5), ('how to make money fast?', 4.1060192584991455, 0.668), ('Netflix', 0.09330344200134277, 0.572), ('Apple computer', 1.1116185188293457, 0.629), ('The Simpsons', 0.03552579879760742, 0.185), ('World cup', 7.760331869125366, 0.167), ('How to lose weight?', 0.6944315433502197, 0.494), ('Java', 0.10708117485046387, 0.439), ('Air Jordan', 1.9328842163085938, 0.495), ('how to deal with depression?', 0.9405021667480469, 0.233), ('How do you make gold', 3.388866662979126, 0.038), ('Marijuana', 0.0329082012176

In [82]:
# anchor w = 1.6 with question words body_w *2
avg = sum(x[2] for x in qs_res)/len(qs_res)
print(avg)
avg_duration = sum(x[1] for x in qs_res)/len(qs_res)
print(avg_duration)
print(qs_res)

0.46166666666666667
2.5382814168930055
[('best marvel movie', 5.404267072677612, 0.433), ('How do kids come to world?', 6.631540060043335, 0.205), ('Information retrieval', 1.2719502449035645, 0.913), ('LinkedIn', 0.016895532608032227, 0.879), ('How to make coffee?', 2.4430606365203857, 0.233), ('Ritalin', 0.0009758472442626953, 0.862), ('How to make wine at home?', 5.543442249298096, 0.34), ('Most expensive city in the world', 10.23403000831604, 0.293), ('India', 1.1919646263122559, 0.5), ('how to make money fast?', 4.147023439407349, 0.697), ('Netflix', 0.09278988838195801, 0.572), ('Apple computer', 1.1095292568206787, 0.629), ('The Simpsons', 0.03660869598388672, 0.185), ('World cup', 7.753111839294434, 0.167), ('How to lose weight?', 0.7032713890075684, 0.461), ('Java', 0.11136054992675781, 0.439), ('Air Jordan', 1.972252607345581, 0.495), ('how to deal with depression?', 0.937619686126709, 0.227), ('How do you make gold', 3.3466265201568604, 0.0), ('Marijuana', 0.0314865112304687

In [69]:
# anchor w = 1.5
avg = sum(x[2] for x in qs_res)/len(qs_res)
print(avg)
avg_duration = sum(x[1] for x in qs_res)/len(qs_res)
print(avg_duration)
print(qs_res)

0.4616666666666667
2.5252612193425494
[('best marvel movie', 5.38534951210022, 0.421), ('How do kids come to world?', 6.549575090408325, 0.196), ('Information retrieval', 1.2789475917816162, 0.901), ('LinkedIn', 0.017145395278930664, 0.879), ('How to make coffee?', 2.425619125366211, 0.239), ('Ritalin', 0.0009198188781738281, 0.862), ('How to make wine at home?', 5.48232889175415, 0.353), ('Most expensive city in the world', 10.223955154418945, 0.43), ('India', 1.1905243396759033, 0.5), ('how to make money fast?', 4.097115993499756, 0.69), ('Netflix', 0.0889897346496582, 0.587), ('Apple computer', 1.1069245338439941, 0.647), ('The Simpsons', 0.03663754463195801, 0.185), ('World cup', 7.771716833114624, 0.167), ('How to lose weight?', 0.7082512378692627, 0.45), ('Java', 0.11243700981140137, 0.439), ('Air Jordan', 1.9405205249786377, 0.363), ('how to deal with depression?', 0.9405269622802734, 0.21), ('How do you make gold', 3.3814380168914795, 0.0), ('Marijuana', 0.032479047775268555, 0

In [35]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list): 
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
#     print("recall: " + len(precisions)+" out of 40")
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [130]:
print(queries["Winter"])

[1511596, 52709838, 1673945, 17349106, 8521120, 3060382, 16615604, 8438818, 8351234, 979072, 64928991, 36439749, 1817908, 1971153, 30276826, 4538366, 1298502, 22933429, 6511088, 38950, 19431459, 1221144, 19938267, 1843684, 34069, 6201653, 33672235, 3548574, 1372169, 65601132, 38416091, 1088531, 2020857, 316711, 34061, 9825536, 22190045, 43343961, 1632099, 962053, 961505]


In [128]:
def get_ap(q_title, weights_list):
    res = search(q_title ,weights)
    predicted, _ = zip(*res)
    actual = queries[q_title]
    ap = average_precision(actual, predicted)
    return ap

q_title = "World Cup 2022"
# weights = [0.75133306, 0.08561443, 0.74563351, 0.97631322, 0.54191462]
# Winter query weights: 
weights = [0.55754871, 1.25785412, 0.54434859, 0.09795087, 0.38754463]
print(get_ap(q_title, weights))
# actual = [54046846, 49170369, 55339286, 62417830, 54251265, 41283158, 65819511, 49029294, 67520032, 49134382, 42311608, 63656330, 52261594, 41185040, 43794572, 64413225, 49131135, 49127974, 43794574, 63656361, 49128142, 41699729, 51759111, 47762921, 55708102, 63656365, 55339303, 57390230, 67830379, 61805032, 26091326, 54802759]
# weights = [0.75133306, 0.08561443, 0.74563351, 0.97631322, 0.54191462]
# res = search('Rick and Morty' ,weights)
# predicted, _ = zip(*res)
# ap = average_precision(actual, predicted)
# print(ap)

TypeError: search() takes 1 positional argument but 2 were given

In [63]:
# def queries_ap(weights, q_num=1):
#     queries = list(queries.items())[:q_num]
#     qs_res = []

#     for q, true_wids in queries:
#         duration, ap = None, None
#         t_start = time()
#         pred_wids = search(q, weights)
#         duration = time() - t_start
#         ap = average_precision(true_wids, pred_wids)
#         qs_res.append((q, duration, ap, weights))
#     return qs_res

In [129]:
def queries_ap(weights, query):
#     queries = list(queries.items())[:q_num]
#     qs_res = []
    queries = [query]
    for q, true_wids in queries:
        ap = None
#         -----------------------------------------------------
        res = search(q, weights)
        pred_wids, _ = zip(*res)
        ap = average_precision(true_wids, pred_wids)
#         qs_res.append((q, duration, ap, weights))
    return ap

In [113]:

# query_to_run = (q_title, queries[q_title])
# print(query_to_run)

('Winter', [1511596, 52709838, 1673945, 17349106, 8521120, 3060382, 16615604, 8438818, 8351234, 979072, 64928991, 36439749, 1817908, 1971153, 30276826, 4538366, 1298502, 22933429, 6511088, 38950, 19431459, 1221144, 19938267, 1843684, 34069, 6201653, 33672235, 3548574, 1372169, 65601132, 38416091, 1088531, 2020857, 316711, 34061, 9825536, 22190045, 43343961, 1632099, 962053, 961505])


In [125]:
import numpy as np
from scipy.optimize import minimize

In [119]:


def print_optimized_weights(q_title):
    query_to_run = (q_title, queries[q_title])
#     print(query_to_run)
    # Define the function to be maximized
    def y(x):
        x1, x2, x3, x4, x5 = x
        weights = x.tolist()
        res =  queries_ap(weights,query_to_run)
        return -1 * res

    # Define the initial point
    weights = [0.6, 1.6, 1.25, 1.8, 1.3]
    x0 = np.matrix(weights)
    
#     x0 = np.random.uniform(0, 1, 5)

    # Number of iterations
    num_iterations = 10

    # list to hold the best points
    best_points = []

    # Perform the optimization num_iterations times
    for i in range(num_iterations):
        # Optimize the function
        bounds = [(0, None)]*5

        result = minimize(y, x0, bounds=bounds, method='L-BFGS-B')
        # add the point to the best points list
        best_points.append((-result.fun, result.x))
        x0 = result.x
        # Randomize x0 a little bit to avoid getting stuck in a local minimum
        x0 += np.random.normal(scale=0.1, size=5)
#         print(x0)
    # Sort the list of best points and take the top 10
    best_points.sort(key= lambda y_x: y_x[0] ,reverse=True)
    top_10 = best_points[:5]

    # Print the 10 best points
    print("anchor, title, body, page_rank, page_views")
    for i, point in enumerate(top_10):
        print("{}. y = {}, x = {}".format(i+1, point[0], point[1]))


In [118]:
q_title = "Rick and Morty"
print_optimized_weights(q_title)

[0.72003659 0.62859251 0.41320103 0.64153128 0.45821703]
[0.70467526 0.67284511 0.50217344 0.63171468 0.40092812]
[0.73573968 0.73870666 0.52359982 0.61458126 0.48539368]
[0.53134471 0.84669479 0.49451533 0.46530829 0.5732673 ]
[0.6462671  0.75457893 0.61579258 0.64595072 0.55396991]
[0.69688303 0.6558302  0.57247377 0.58757528 0.73187791]
[0.70745614 0.70524924 0.64024859 0.53248478 0.7185971 ]
[0.77795504 0.66104408 0.76544221 0.65682795 0.74708016]
[0.66565863 0.71939317 0.85524436 0.6167213  0.74657466]
[0.5662838  0.67381974 0.78139254 0.5649084  0.72723188]
anchor, title, body, page_rank, page_views
1. y = 0.818, x = [0.66565863 0.71939317 0.85524436 0.6167213  0.74657466]
2. y = 0.816, x = [0.77795504 0.66104408 0.76544221 0.65682795 0.74708016]
3. y = 0.813, x = [0.5662838  0.67381974 0.78139254 0.5649084  0.72723188]
4. y = 0.81, x = [0.70745614 0.70524924 0.64024859 0.53248478 0.7185971 ]
5. y = 0.807, x = [0.6462671  0.75457893 0.61579258 0.64595072 0.55396991]
6. y = 0.802,

In [121]:
q_title = "Winter"
print_optimized_weights(q_title)

anchor, title, body, page_rank, page_views
1. y = 0.751, x = [ 0.3802622   1.15174287  0.66534942  0.04572521 -0.02795971]
2. y = 0.736, x = [0.55754871 1.25785412 0.54434859 0.09795087 0.38754463]
3. y = 0.718, x = [ 0.70739086  1.01978448  0.34099951 -0.09697754  0.25905376]
4. y = 0.712, x = [ 0.47024579  1.0151511   0.31649317 -0.00362412  0.52768426]
5. y = 0.63, x = [0.50676697 1.00784559 0.3411947  0.0340439  0.65066275]
6. y = 0.618, x = [0.44357598 1.0101824  0.14261672 0.0160672  0.88657244]
7. y = 0.508, x = [0.5154044  1.01465605 0.57672439 0.05566338 0.09372452]
8. y = 0.507, x = [ 0.52229925  0.92198258 -0.05646227  0.1800847   0.84241589]
9. y = 0.491, x = [0.52162412 1.02846453 0.07928739 0.06320404 0.73917427]
10. y = 0.373, x = [0.52621377 1.17490465 0.58755963 0.07533695 0.09622741]


In [123]:

def print_optimized_weights_all_queries(queries):
#     query_to_run = (q_title, queries[q_title])
#     print(query_to_run)
    # Define the function to be maximized
    results_dict = {}
    
    def y(x):
        ap_list = []
        for query_to_run in queries.items():
            x1, x2, x3, x4, x5 = x
            weights = x.tolist()
            ap =  queries_ap(weights,query_to_run)
            ap_list.append(ap)
        results_dict[(x1, x2, x3, x4, x5)]=ap_list 
        res = sum(ap_list)/len(ap_list)
        return -1 * res

    # Define the initial point
    weights = [0.6, 1.6, 1.25, 1.8, 1.3]
    x0 = np.matrix(weights)

    # Number of iterations
    num_iterations = 8

    # list to hold the best points
    best_points = []

    # Perform the optimization num_iterations times
    for i in range(num_iterations):
        # Optimize the function
        bounds = [(0, None)]*5

        result = minimize(y, x0, bounds=bounds, method='L-BFGS-B')
        # add the point to the best points list
        best_points.append((-result.fun, result.x))
        x0 = result.x
        # Randomize x0 a little bit to avoid getting stuck in a local minimum
        x0 += np.random.normal(scale=0.1, size=5)
        print(f"finished iteration: {i}")
    # Sort the list of best points and take the top 10
    best_points.sort(key= lambda y_x: y_x[0] ,reverse=True)
    top_10 = best_points[:5]

    # Print the 10 best points
    print("anchor, title, body, page_rank, page_views")
    for i, point in enumerate(top_10):
        print("{}. y = {}, x = {}".format(i+1, point[0], point[1]))
    
    print("----------------------------------------------------------------")
    print(results_dict)


In [47]:
# queries_partial = list(queries_train.items())[:2]
# queries = {}
# for item in queries_partial:
#     queries[item[0]]= item[1]
# print(queries)


len(queries_train)

20

In [ ]:
queries = queries_train
print_optimized_weights_all_queries(queries)

finished iteration: 0
finished iteration: 1
finished iteration: 2
finished iteration: 3
finished iteration: 4
finished iteration: 5
finished iteration: 6
finished iteration: 7
finished iteration: 8
finished iteration: 9
anchor, title, body, page_rank, page_views
1. y = 0.44435000000000013, x = [0.8651133  0.8916274  0.20168009 0.38727421 0.4619754 ]
2. y = 0.44305000000000005, x = [0.83487311 0.88377172 0.19311083 0.56437264 0.6301511 ]
3. y = 0.44275000000000003, x = [0.97429313 0.78071818 0.25346919 0.30582536 0.50987861]
4. y = 0.44255000000000005, x = [0.84636032 0.76538511 0.24043806 0.51229229 0.75173852]
5. y = 0.4418000000000001, x = [0.89516389 0.77604771 0.12645326 0.5439087  0.50756813]
----------------------------------------------------------------
{(0.7757580409651665, 0.7558793562200189, 0.13275155763522895, 0.4389014498383995, 0.34578515655389963): [0.437, 0.42, 0.779, 0.81, 0.246, 0.839, 0.319, 0.283, 0.281, 0.848, 0.572, 0.564, 0.123, 0.49, 0.381, 0.402, 0.662, 0.164,

In [ ]:
print_optimized_weights_all_queries(new_queries)

/tmp/ipykernel_13307/163962462.py:33: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(y, x0, bounds=bounds, method='L-BFGS-B')


finished iteration: 0
finished iteration: 1
finished iteration: 2
finished iteration: 3
finished iteration: 4
finished iteration: 5
finished iteration: 6
finished iteration: 7
anchor, title, body, page_rank, page_views
1. y = 0.47513333333333324, x = [0.46162036 1.42513415 1.13945286 1.87296152 1.39693174]
2. y = 0.47203333333333336, x = [0.56177509 1.57824883 1.11656287 1.77349953 1.30673648]
3. y = 0.47049999999999986, x = [0.59837057 1.40591699 1.34453699 1.82661049 1.28270854]
4. y = 0.46893333333333326, x = [0.65537442 1.32218089 1.09079092 2.0588799  1.43638221]
5. y = 0.4677999999999999, x = [0.41127339 1.39195664 1.22941912 1.86734223 1.37474593]
----------------------------------------------------------------
{(0.6, 1.6, 1.25, 1.8, 1.3): [0.433, 0.205, 0.913, 0.879, 0.233, 0.862, 0.34, 0.293, 0.5, 0.697, 0.572, 0.629, 0.185, 0.167, 0.461, 0.439, 0.495, 0.227, 0.333, 0.33, 0.582, 0.339, 0.855, 0.825, 0.418, 0.874, 0.142, 0.0, 0.433, 0.5], (0.60000001, 1.6, 1.25, 1.8, 1.3): [0.4

In [133]:
!ls

 Instance_creation.ipynb     page_views_dict.pkl
 __pycache__		     postings_gcp_anchor
 anchor_index.pkl	     postings_gcp_body
 body_index.pkl		     postings_gcp_title
 doc_id_title_dict.pickle   'queries control group.json'
 doc_len_dict.pickle	     query_expansion_dict.pickle
 doc_norm_dict.pickle	     search_backend.py
 download_to_gcp.ipynb	     search_frontend.py
 index_creation_body.ipynb   startup_script_gcp.sh
 inverted_index_gcp.py	     test_frontend.ipynb
 new_train.json		     test_queries.json
 page_rank_creation.ipynb    title_index.pkl
 page_rank_dict.pickle


In [59]:
def mean_ap_on_test_set(queries, weights):
    ap_list = []
    for query_to_run in queries.items():
#         x1, x2, x3, x4, x5 = weights
#         weights = x.tolist()
        ap =  queries_ap(weights,query_to_run)
        ap_list.append(ap)
    print(ap_list)
    res = sum(ap_list)/len(ap_list)
    print(res)
    return res

In [66]:
weights_test = [0.55754871, 1.25785412, 0.54434859, 0.09795087, 0.38754463]
mean_ap_on_test_set(queries_control_group, weights_test)

[0.575, 0.395, 0.743, 0.891, 0.276, 0.884, 0.484, 0.1, 0.481, 0.111]
0.4939999999999999


0.4939999999999999

In [22]:
with open('query_expansion_dict.pickle', 'rb') as f:
    # global query_expansion_dict
    query_expansion_dict = pickle.load(f)

In [26]:
def query_expansion(tokenized_query, k):
    if k <= 0:
        return tokenized_query
    # tokenized_query = tokenize(query)
    expansion_list = []
    for token in tokenized_query:
        token_expansion = query_expansion_dict.get(token, None)
        tmp = k
        if not token_expansion:
            continue
        for i in range(len(token_expansion)):
            if tmp <= 0:
                break
            expansion_list.append(token_expansion[i][1])
            tmp -= 1
        
    tokenized_query.update(expansion_list)
    return tokenized_query

In [33]:
    query = "netflix"
    tokenized_query = tokenize(query)
    q_len = len(tokenized_query)
    expand_factor = 0
    if q_len == 1:
        expand_factor = 2
    elif q_len == 2 or q_len == 3:
        expand_factor = 1
    else:
        expand_factor = 0
    tokens = query_expansion(tokenized_query, expand_factor)
    print(tokens)

{'chromecast', 'netflix', 'hulu'}


In [29]:
for query_before in queries_train.keys(): 
    print(query_before)
    tokenized_query = tokenize(query_before)
    q_len = len(tokenized_query)
    expand_factor = 0
    if q_len == 1:
        expand_factor = 2
    elif q_len == 2 or q_len == 3:
        expand_factor = 1
    else:
        expand_factor = 0
    query_after = query_expansion(tokenized_query, expand_factor)
    print(query_after)

best marvel movie
{'film', 'best', 'movie', 'marvels', 'good', 'marvel'}
How do kids come to world?
{'world', 'come', 'planet', 'get', 'kids', 'toddlers'}
Information retrieval
{'information', 'processing', 'details', 'retrieval'}
LinkedIn
{'linkedin'}
How to make coffee?
{'coffee', 'make', 'give', 'espresso'}
Ritalin
{'ritalin'}
How to make wine at home?
{'home', 'make', 'house', 'truffle', 'give', 'wine'}
Most expensive city in the world
{'world', 'expensive', 'city', 'planet', 'cheaper', 'town'}
India
{'pakistan', 'gujarat', 'india'}
how to make money fast?
{'fast', 'cash', 'money', 'slow', 'make', 'give'}
Netflix
{'chromecast', 'netflix', 'hulu'}
Apple computer
{'computers', 'apple', 'blackberry', 'computer'}
The Simpsons
{'simpsons', 'godzilla', 'futurama'}
World cup
{'cups', 'world', 'cup', 'planet'}
How to lose weight?
{'weight', 'lose', 'regain', 'weights'}
Java
{'php', 'java', 'api'}
Air Jordan
{'jordans', 'air', 'heat', 'jordan'}
how to deal with depression?
{'depression', 'd

In [32]:
print(query_expansion_dict['netflix'])

[(0.8382608294487, 'hulu'), (0.8235220909118652, 'chromecast'), (0.7977921366691589, 'redbox'), (0.7907706499099731, 'xbox'), (0.7830523252487183, 'itunes')]


In [35]:
anchor_index.df['hulu']

57

In [55]:
weights = [0.55754871, 1.25785412, 0.54434859, 0.09795087, 0.38754463]
BODY_WEIGHT, ANCHOR_WEIGHT, TITLE_WEIGHT, PAGE_RANK_WEIGHT, PAGE_VIEW_WEIGHT = weights
res = []
similarity_dict = {}

tokens = {'chromecast', 'netflix', 'hulu'}
# tokens={'netflix'}
# tokens={'netflix', 'hulu'}
for token in tokens:

    # Searching the term in anchor index
    if anchor_index.df.get(token, 0) != 0:
        for doc_tf in anchor_index.read_posting_list(token):
            # token_doc_anchor_occurrences[doc_tf[0]] = 1 + token_doc_anchor_occurrences.get(doc_tf[0], 0)
            similarity_dict[doc_tf[0]] = similarity_dict.get(doc_tf[0], 0) + ANCHOR_WEIGHT

    # Searching the term in title index
    if title_index.df.get(token, 0) != 0:
        for doc_tf in title_index.read_posting_list(token):
            similarity_dict[doc_tf[0]] = similarity_dict.get(doc_tf[0], 0) + TITLE_WEIGHT

    # Searching the term in body index
    if body_index.df.get(token, 0) != 0:
        for doc_tf in body_index.read_posting_list(token):
            c = bm25_update(token, doc_tf[0], doc_tf[1]) * BODY_WEIGHT
            similarity_dict[doc_tf[0]] = similarity_dict.get(doc_tf[0], 0) + c

    # Adding page_rank and page_views of each relevant document
    for doc_id in similarity_dict.keys():
        similarity_dict[doc_id] += page_rank_dict.get(doc_id, 0) / page_rank_max * PAGE_RANK_WEIGHT
        similarity_dict[doc_id] += page_views_dict.get(doc_id, 0) / page_views_max * PAGE_VIEW_WEIGHT

top_n_results(similarity_dict, res, 100)

print(res)

In [50]:
%cd /home/dataproc
with open('new_train.json', 'rt') as f:
    queries_new = json.load(f)

/home/dataproc


In [54]:
average_precision(queries_new, res, k=100)

0.0

In [ ]:
#Training bm25
def bm25_update(token, doc_id, tf):
    k1 = 1.2
    B = 0.75
    N = 6348910
    avgl = 319.5242353411845
    idf = math.log(((N - body_index.df[token] + 0.5)/(body_index.df[token]+0.5)) + 1, math.e)
    score = (idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - B + B * (doc_len_dict[doc_id] / avgl)))))
    return score

